In [ ]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration

In [ ]:
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np

In [ ]:
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    return transcriber({"sampling_rate": sr, "raw": y})["text"]

In [ ]:
import base64
import soundfile
import json
import io
import requests

host = "http://localhost:4111"
text = "I am voice number 22. I sound more stable than voice number 99"
sid = 22

def tts(text: str):
    resp = requests.post(host + '/tts', data={
        "text": text,
        "sid": sid
    })

    audiob64 = json.loads(resp.text)['audio']
    audio_bytes = base64.b64decode(audiob64)
    audio_file = io.BytesIO(audio_bytes)
    audio_file.seek(0)
    audio, sr = soundfile.read(audio_file)

    return sr, audio

In [ ]:
import gradio as gr

demo = gr.Interface(
    tts,
    gr.Text(),
    "audio"
)

demo.launch()

In [ ]:
import os
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain

os.environ["OPENAI_API_KEY"] = "api_key"


template = """
USER: You are a helpful, medical specialist. Always answer as helpfully as possible.  If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. You always answer medical questions based on facts.
ASSISTANT: Ok great ! I am a medical expert!
USER: {question}
ASSISTANT:
"""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm = OpenAI(openai_api_base="http://localhost:3000/v1")
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What can I do about glenoid cavity injury ?"

result = llm_chain.run(question)
print(result)

In [1]:
import os
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain
import intent

# prompt = PromptTemplate(template=template, input_variables=["question"])
# llm = OpenAI(openai_api_base="https://api.openai.com/v1")
# llm_chain = LLMChain(prompt=prompt, llm=llm)

# question = "What's the weather in Ottawa"

# result = llm_chain.run(question)
# print(result)

In [ ]:
import intent

prompt = PromptTemplate(template=intent.intents['check-weather']['response-template'], input_variables=["location", "temperature",])
llm = OpenAI(openai_api_base="https://api.openai.com/v1")
llm_chain = LLMChain(prompt=prompt, llm=llm)

result = llm_chain.run(location='Ottawa', temperature='20C')
print(result)

In [ ]:
prompt = PromptTemplate(template=intent.intents['open-app']['response-template'], input_variables=["app","open_result",])
llm = OpenAI(openai_api_base="https://api.openai.com/v1")
llm_chain = LLMChain(prompt=prompt, llm=llm)

result = llm_chain.run(app="discord", open_result="Opened")
print(result)

In [ ]:
from intent import template

print(template)

In [ ]:
import dotenv
dotenv.load_dotenv()
import os
os.environ['OPENAI_API_KEY']

In [ ]:
import os
import dotenv
dotenv.load_dotenv(override=True)
api_key = os.environ['OPENAI_API_KEY']

print(api_key)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

In [ ]:
import re

p = re.compile('User Intent:.*')
lp = re.compile('location:.*')
s = '''User Intent:check-weather
Slots:
location:Ottawa'''

matches = p.search(s)
print(matches)
matches = lp.search(s)
print(matches)